In [133]:
from scipy.stats import linregress
import numpy as np
import pandas as pd
import datetime as dt
import sklearn
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [134]:
# Call in portion of data for basic manipulation
df = pd.read_csv('./Data/data.csv')
df.head()
# df.describe()

,Date,Rot,VH,Team,1st,2nd,Final,Open,Close,ML,2H
0,1109,711,V,FloridaIntl,30,42,72,146,150.5,10000,79
1,1109,712,H,NorthCarolina,46,42,88,30,30.5,-30000,14.5
2,1109,713,V,MurrayState,26,44,70,132,139.5,700,73
3,1109,714,H,California,34,41,75,13,13.5,-1100,6
4,1109,715,V,AlcornState,28,32,60,NL,NL,16000,NL


In [135]:
#finding outlier games/teams
count_df = df['Team'].value_counts()
count_dict = count_df.to_dict()
df['Team Frequency'] = df['Team'].map(count_dict)
# df.head()
# df.describe()
# team_list = df['Team'].unique()
# list_df = pd.DataFrame(team_list)
# list_df.to_csv('team_list', sep=',', encoding='utf-8', index=False)

In [136]:
# encode the team names for easier training/testing
le = LabelEncoder()
le.fit(df['Team'])
team_code = le.transform(df['Team'])
df['Team Code'] = team_code
df.head()

,Date,Rot,VH,Team,1st,2nd,Final,Open,Close,ML,2H,Team Frequency,Team Code
0,1109,711,V,FloridaIntl,30,42,72,146,150.5,10000,79,270,107
1,1109,712,H,NorthCarolina,46,42,88,30,30.5,-30000,14.5,382,232
2,1109,713,V,MurrayState,26,44,70,132,139.5,700,73,313,211
3,1109,714,H,California,34,41,75,13,13.5,-1100,6,344,46
4,1109,715,V,AlcornState,28,32,60,NL,NL,16000,NL,78,9


In [137]:
# Transferring home/away status to numeric
def home_away_func(row):
    if row['VH'] == 'H':
        return 1
    elif row['VH'] == 'V':
        return 0
    else:
        return 0.5

In [138]:
df['Home/Away/Neutral'] = df.apply(home_away_func, axis=1)
# df.head()

In [185]:
track_df = pd.DataFrame(columns=['Team','Code'])


track_df['Team'] = df['Team'].unique()
track_df['Code'] = df['Team Code'].unique()

team_dict = dict(zip(track_df.Team, track_df.Code))
# area_dict = dict(zip(lakes.area, lakes.count))
print(team_dict.get('Kentucky', None))
print(team_dict.get('Tennessee', None))


159
332


In [139]:
coded_df = pd.get_dummies(df["Team Code"])
# coded_df.nunique()

In [140]:
# Concatenate the coded dataframe to the end of the existing, base dataframe
concat_df = pd.concat([df, coded_df],axis=1)

In [ ]:
# Create dictionary of team names, team codes, and one hot encoded id arrays
# team_dict = 

In [141]:
# Reorder to put entire game data on single row
odd = concat_df.iloc[::2]
even = concat_df.iloc[1::2]

In [142]:
merged_df = odd.reset_index(drop=True).merge(even.reset_index(drop=True), left_index=True, right_index=True)
merged_df.head()

,Date_x,Rot_x,VH_x,Team_x,1st_x,2nd_x,Final_x,Open_x,Close_x,ML_x,...,395_y,396_y,397_y,398_y,399_y,400_y,401_y,402_y,403_y,404_y
0,1109,711,V,FloridaIntl,30,42,72,146,150.5,10000,...,0,0,0,0,0,0,0,0,0,0
1,1109,713,V,MurrayState,26,44,70,132,139.5,700,...,0,0,0,0,0,0,0,0,0,0
2,1109,715,V,AlcornState,28,32,60,NL,NL,16000,...,0,0,0,0,0,0,0,0,0,0
3,1109,717,V,AlbanyNY,20,23,43,NL,NL,1500,...,0,0,0,0,0,0,0,0,0,0
4,1111,727,V,DetroitU,32,29,61,142,140.5,1350,...,0,0,0,0,0,0,0,0,0,0


In [143]:
cleaned_df = merged_df.rename(columns = {'Date_x':'Date','VH_x':'Loc Team X','Team_x':'Team X','1st_x':'Team X 1st Half',
                                      '2nd_x':'Team X 2nd Half','Final_x':'Team X Final','VH_y':'Loc Team Y',
                                      'Team_y':'Team Y','1st_y':'Team Y 1st Half','2nd_y':'Team Y 2nd Half',
                                     'Final_y':'Team Y Final','Open_x':'Total Open','Close_x':'Total Close','ML_x':'Team X Moneyline',
                                        '2H_x':'2nd Half Total','Open_y':'Spread Open','Close_y':'Spread Close','ML_y':'Team Y Moneyline',
                                        '2H_y':'2nd Half Spread'})
cleaned_df.head()

################################################################################################################
### Drop columns for total open and spread open for simplicity sake
### Drop all games with "NL" and replace "pk" with 0
### Rewrite the columns to flip the spread and totals when the 'spread' column is greater than the 'total' column.
### Then flip the direction to negative thereby making the home favorite a positive value and the home dog a negative value
################################################################################################################

,Date,Rot_x,Loc Team X,Team X,Team X 1st Half,Team X 2nd Half,Team X Final,Total Open,Total Close,Team X Moneyline,...,395_y,396_y,397_y,398_y,399_y,400_y,401_y,402_y,403_y,404_y
0,1109,711,V,FloridaIntl,30,42,72,146,150.5,10000,...,0,0,0,0,0,0,0,0,0,0
1,1109,713,V,MurrayState,26,44,70,132,139.5,700,...,0,0,0,0,0,0,0,0,0,0
2,1109,715,V,AlcornState,28,32,60,NL,NL,16000,...,0,0,0,0,0,0,0,0,0,0
3,1109,717,V,AlbanyNY,20,23,43,NL,NL,1500,...,0,0,0,0,0,0,0,0,0,0
4,1111,727,V,DetroitU,32,29,61,142,140.5,1350,...,0,0,0,0,0,0,0,0,0,0


In [144]:
cleaned_df.drop(cleaned_df[cleaned_df['Total Close'] == 'NL'].index, inplace = True) 
cleaned_df.drop(cleaned_df[cleaned_df['Spread Close'] == 'NL'].index, inplace=True)
cleaned_df.drop(cleaned_df[cleaned_df['2nd Half Spread'] == 'NL'].index, inplace=True)
cleaned_df.drop(cleaned_df[cleaned_df['2nd Half Total'] == 'NL'].index, inplace=True)
cleaned_df.drop(cleaned_df[cleaned_df['Team X Moneyline'] == 'NL'].index, inplace=True)
cleaned_df.drop(cleaned_df[cleaned_df['Team Y Moneyline'] == 'NL'].index, inplace=True)

In [145]:
# indexNames = temp_df[(temp_df['Total Close']=="NL") & (temp_df['Spread Close']=="NL")].index
# temp2_df = temp_df.drop(indexNames, inplace=True)
cleaned_df.drop(columns=['Total Open','Spread Open'])
cleaned_df.head()

,Date,Rot_x,Loc Team X,Team X,Team X 1st Half,Team X 2nd Half,Team X Final,Total Open,Total Close,Team X Moneyline,...,395_y,396_y,397_y,398_y,399_y,400_y,401_y,402_y,403_y,404_y
0,1109,711,V,FloridaIntl,30,42,72,146,150.5,10000,...,0,0,0,0,0,0,0,0,0,0
1,1109,713,V,MurrayState,26,44,70,132,139.5,700,...,0,0,0,0,0,0,0,0,0,0
4,1111,727,V,DetroitU,32,29,61,142,140.5,1350,...,0,0,0,0,0,0,0,0,0,0
7,1112,505,V,JamesMadison,25,19,44,149,140.5,1500,...,0,0,0,0,0,0,0,0,0,0
8,1112,507,V,GeorgiaState,29,24,53,137.5,135,450,...,0,0,0,0,0,0,0,0,0,0


In [146]:
cleaned_df = cleaned_df.loc[(cleaned_df['Team Frequency_x'] > 10) & (cleaned_df['Team Frequency_y'] > 10)]

In [147]:
# Find winner
def win_func(row):
    if row['Team X Final'] > row['Team Y Final']:
        return row['Team X']
    else:
        return row['Team Y']

cleaned_df['Winner'] = cleaned_df.apply(win_func, axis=1)

In [148]:
# Drop meaningless columns
shortened_df = cleaned_df.drop(columns=['Rot_x','Date_y','Rot_y','Total Open','Spread Open'])

In [149]:
# Convert objects to floats
shortened_df['Total Close'] = pd.to_numeric(shortened_df['Total Close'])
shortened_df['Spread Close'] = pd.to_numeric(shortened_df['Spread Close'])
shortened_df['2nd Half Spread'] = pd.to_numeric(shortened_df['2nd Half Spread'])
shortened_df['2nd Half Total'] = pd.to_numeric(shortened_df['2nd Half Total'])
shortened_df['Team X Moneyline'] = pd.to_numeric(shortened_df['Team X Moneyline'])
shortened_df['Team Y Moneyline'] = pd.to_numeric(shortened_df['Team Y Moneyline'])

In [150]:
###### Use the df.apply to create corrected columns then drop bad column
def spread_func(row):
    if row['Total Close'] < row['Spread Close']:
        return row['Total Close']
    else:
        return (row['Spread Close'] * -1)
    
def total_func(row):
    if row['Total Close'] < row['Spread Close']:
        return row['Spread Close']
    else:
        return row['Total Close']

In [151]:
shortened_df['Corrected Spread'] = shortened_df.apply(spread_func, axis=1)
shortened_df['Corrected Total'] = shortened_df.apply(total_func, axis=1)
shortened_df.head()

,Date,Loc Team X,Team X,Team X 1st Half,Team X 2nd Half,Team X Final,Total Close,Team X Moneyline,2nd Half Total,Team Frequency_x,...,398_y,399_y,400_y,401_y,402_y,403_y,404_y,Winner,Corrected Spread,Corrected Total
0,1109,V,FloridaIntl,30,42,72,150.5,10000,79.0,270,...,0,0,0,0,0,0,0,NorthCarolina,-30.5,150.5
1,1109,V,MurrayState,26,44,70,139.5,700,73.0,313,...,0,0,0,0,0,0,0,California,-13.5,139.5
4,1111,V,DetroitU,32,29,61,140.5,1350,73.5,326,...,0,0,0,0,0,0,0,California,-17.0,140.5
7,1112,V,JamesMadison,25,19,44,140.5,1500,72.5,312,...,0,0,0,0,0,0,0,OhioState,-19.0,140.5
8,1112,V,GeorgiaState,29,24,53,135.0,450,71.0,322,...,0,0,0,0,0,0,0,NCState,-10.5,135.0


In [152]:
# shortened_df.dtypes
### Create function to see who wins against the spread
### if they cover, they get a 1, if no cover or push, then 0
def team_x_cover(row):
    if (row['Team Y Final'] + row['Corrected Spread']) < row['Team X Final']:
        return 1
    else:
        return 0
    
def team_y_cover(row):
    if (row['Team Y Final'] + row['Corrected Spread']) > row['Team X Final']:
        return 1
    else:
        return 0
    
def ats_winner(row):
    if row['Team X Cover'] > row['Team Y Cover']:
        return row['Team X']
    elif row['Team X Cover'] < row['Team Y Cover']:
        return row['Team Y']
    else:
        return 'Push'
    
def ats_winner_code(row):
    if row['Team X Cover'] > row['Team Y Cover']:
        return row['Team Code_x']
    elif row['Team X Cover'] < row['Team Y Cover']:
        return row['Team Code_y']
    else:
        return 0.5

In [153]:
shortened_df['Team X Cover'] = shortened_df.apply(team_x_cover, axis=1)
shortened_df['Team Y Cover'] = shortened_df.apply(team_y_cover, axis=1)
shortened_df['ATS Winner'] = shortened_df.apply(ats_winner, axis=1)
shortened_df['ATS Winner Code'] = shortened_df.apply(ats_winner_code, axis=1)

In [154]:
# Check output
final_df = shortened_df
final_df.head(50)

,Date,Loc Team X,Team X,Team X 1st Half,Team X 2nd Half,Team X Final,Total Close,Team X Moneyline,2nd Half Total,Team Frequency_x,...,402_y,403_y,404_y,Winner,Corrected Spread,Corrected Total,Team X Cover,Team Y Cover,ATS Winner,ATS Winner Code
0,1109,V,FloridaIntl,30,42,72,150.5,10000,79.0,270,...,0,0,0,NorthCarolina,-30.5,150.5,1,0,FloridaIntl,107.0
1,1109,V,MurrayState,26,44,70,139.5,700,73.0,313,...,0,0,0,California,-13.5,139.5,1,0,MurrayState,211.0
4,1111,V,DetroitU,32,29,61,140.5,1350,73.5,326,...,0,0,0,California,-17.0,140.5,0,1,California,46.0
7,1112,V,JamesMadison,25,19,44,140.5,1500,72.5,312,...,0,0,0,OhioState,-19.0,140.5,0,1,OhioState,251.0
8,1112,V,GeorgiaState,29,24,53,135.0,450,71.0,322,...,0,0,0,NCState,-10.5,135.0,0,1,NCState,215.0
9,1113,V,CSNorthridge,20,43,63,142.0,2000,74.5,295,...,0,0,0,Purdue,-22.5,142.0,0,1,Purdue,269.0
10,1113,V,William&Mary,31,35,66,136.0,2250,73.0,291,...,0,0,0,Connecticut,-23.5,136.0,1,0,William&Mary,393.0
11,1113,V,YoungstownState,24,33,57,137.0,1150,73.0,293,...,1,0,0,Xavier,-16.5,137.0,0,1,Xavier,402.0
12,1113,V,MiamiOhio,42,29,71,1.5,-125,70.0,316,...,0,0,0,Towson,1.5,125.0,0,1,Towson,349.0
13,1113,V,ClevelandState,25,37,62,1.0,-120,3.0,321,...,0,0,0,StBonaventure,1.0,134.0,0,1,StBonaventure,315.0


In [155]:
final_df['ATS Winner'] = final_df['ATS Winner'].astype(str)

In [156]:
final_df['ATS Winner'].nunique()

363

In [157]:
predict_winner_df = final_df.drop(['Date','Team X 1st Half','Team X 2nd Half','Team X Final',
                                  'Total Close','2nd Half Total','Team X','Team Y',
                                   'Team Y 1st Half','Team Y 2nd Half','Team Y Final',
                                   'Spread Close','2nd Half Spread','Winner',
                                  'Loc Team X','Loc Team Y','Team Frequency_x','Team Frequency_y',
                                  'ATS Winner','Team X Cover','Team Y Cover'], axis=1)
predict_winner_df.head()

,Team X Moneyline,Team Code_x,Home/Away/Neutral_x,0_x,1_x,2_x,3_x,4_x,5_x,6_x,...,398_y,399_y,400_y,401_y,402_y,403_y,404_y,Corrected Spread,Corrected Total,ATS Winner Code
0,10000,107,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-30.5,150.5,107.0
1,700,211,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-13.5,139.5,211.0
4,1350,84,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-17.0,140.5,46.0
7,1500,154,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-19.0,140.5,251.0
8,450,119,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-10.5,135.0,215.0


In [158]:
# arranged_cols = predict_winner_df.reindex(columns=['Team Code_x',0,1,2,3,4])

In [159]:
x = predict_winner_df.drop(['ATS Winner Code'], axis=1)
y = predict_winner_df['ATS Winner Code']

print(x.shape, y.shape)

(39349, 818) (39349,)


In [160]:
y.nunique()

363

In [166]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

In [167]:
scalar = StandardScaler()

In [168]:
scalar.fit(x_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [169]:
x_train_scaled = scalar.transform(x_train)

In [170]:
x_test_scaled = scalar.transform(x_test)

In [173]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=818))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=405, activation='softmax'))

In [174]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', #changed from categorical_crossentropy
              metrics=['accuracy'])

In [175]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 100)               81900     
_________________________________________________________________
dense_16 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_17 (Dense)             (None, 405)               40905     
Total params: 132,905
Trainable params: 132,905
Non-trainable params: 0
_________________________________________________________________


In [176]:
model.fit(
    x_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

Train on 29511 samples
Epoch 1/100
29511/29511 - 9s - loss: 2.5362 - acc: 0.3821
Epoch 2/100
29511/29511 - 9s - loss: 1.1721 - acc: 0.5620
Epoch 3/100
29511/29511 - 9s - loss: 1.0128 - acc: 0.6015
Epoch 4/100
29511/29511 - 8s - loss: 0.9376 - acc: 0.6254
Epoch 5/100
29511/29511 - 11s - loss: 0.8873 - acc: 0.6393
Epoch 6/100
29511/29511 - 8s - loss: 0.8578 - acc: 0.6474
Epoch 7/100
29511/29511 - 9s - loss: 0.8248 - acc: 0.6579
Epoch 8/100
29511/29511 - 8s - loss: 0.8031 - acc: 0.6624
Epoch 9/100
29511/29511 - 9s - loss: 0.7772 - acc: 0.6672
Epoch 10/100
29511/29511 - 8s - loss: 0.7547 - acc: 0.6740
Epoch 11/100
29511/29511 - 9s - loss: 0.7362 - acc: 0.6785
Epoch 12/100
29511/29511 - 8s - loss: 0.7158 - acc: 0.6844
Epoch 13/100
29511/29511 - 9s - loss: 0.6926 - acc: 0.6917
Epoch 14/100
29511/29511 - 9s - loss: 0.6761 - acc: 0.6961
Epoch 15/100
29511/29511 - 8s - loss: 0.6600 - acc: 0.7016
Epoch 16/100
29511/29511 - 9s - loss: 0.6401 - acc: 0.7066
Epoch 17/100
29511/29511 - 8s - loss: 0.6

In [177]:
model.save('basketball_trained.h5')